In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer
from sklearn.cluster import KMeans
import seaborn as sns
import plotly.express as px
!pip install --upgrade geopandas matplotlib
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [ ]:
base_grande = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
base_grande.shape

In [ ]:
base_grande.head()

In [ ]:
liste_diff_patho3 = base_grande['patho_niv3'].unique()
liste_diff_patho2 = base_grande['patho_niv2'].unique()
liste_diff_patho1 = base_grande['patho_niv1'].unique()

In [ ]:
#creation de la colonne cancer pour simplifier la base
"""
set_cancer = set()
for i in patho_3_diff : 
    i.split()
    if 'Cancer' in i or 'cancer' in i :
        set_cancer.add(i)
    else :
        continue
"""
base_grande = base_grande.dropna()

In [ ]:
"""
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    base_grande.loc[:,f"{nom_de_maladie}_ind"] = temporaire
"""
#utiliser get_dummies


base_grande = pd.get_dummies(base_grande, prefix = 'indicatrice_', columns=['patho_niv1'], dtype='int')

base_grande["classe_age"] = base_grande["cla_age_5"].apply(
    lambda x: np.mean(list(map(lambda y : int(y), x.split("-")))) if "-" in x else (np.nan if x == 'tsage' else 100)
)


In [ ]:
#on refait ce qu'on avait sur le main pour pouvoir avoir les memes variables,
#la meme taille initiale, les memes individus pour pouvoir voir ou ca coince

In [ ]:
#base_grande['Cancer'] = base_grande[liste_cancer_ind].sum(axis = 1)

In [ ]:
base_grande = base_grande.drop(['top', 'cla_age_5', 'patho_niv2', 'patho_niv3', 'annee', 
                                'libelle_classe_age', 'sexe', 'libelle_sexe'], axis = 1)

In [ ]:
df_model_cancer = base_grande[['dept', 'classe_age', 'indicatrice__Cancers']]

In [ ]:
df_model_cancer.head()

In [ ]:
#https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb

In [ ]:
#debut de la création de carte

In [ ]:
france = gpd.read_file('https://www.data.gouv.fr/fr/datasets/r/90b9341a-e1f7-4d75-a73c-bbc010c7feeb')

In [ ]:
france.plot()

In [ ]:
df_carto = pd.merge(left = df_model_cancer, right = france, 
                    how = 'left', left_on='dept', right_on='code')

In [ ]:
df_carto['dept'].unique()

In [ ]:
dept_manquants = ['999', '974', '971', '972', '973', '976']

In [ ]:
df_carto = df_carto[~df_carto['dept'].isin(dept_manquants)]

In [ ]:
cancer_dep = df_carto.groupby('dept')['indicatrice__Cancers'].sum().reset_index()

In [ ]:
df_carto = df_carto.merge(cancer_dep, on='dept', suffixes=('', '_sum'))

In [ ]:
df_carto = gpd.GeoDataFrame(df_carto, crs='EPSG:4326')

In [ ]:
#fig, ax = plt.subplots(figsize=(10, 10))
#df_carto.plot(ax=ax, column='Cancers_ind_sum', legend=True, cmap='OrRd')
#plt.show()

In [ ]:
df_carto2 = pd.merge(left = france, right = cancer_dep, 
                    how = 'left', left_on='code', right_on='dept')
fig, ax = plt.subplots(figsize=(10, 10))
df_carto2.plot(ax=ax, column='indicatrice__Cancers', legend=True, cmap='OrRd')
plt.show()

In [ ]:
df_carto2

In [ ]:
base_pop = pd.read_csv("base_pop_dep.csv")

base_pop
df_carto2["pourcentage_cancer"] = (df_carto2["indicatrice__Cancers"] / base_pop["total_habitants"])*100

df_carto2

fig, ax = plt.subplots(figsize=(10, 10))
df_carto2.plot(ax=ax, column='pourcentage_cancer', legend=True, cmap='magma')
plt.show()

#effet de population imp? 
#reste a voir avec regression/acp/clustering
#les departements avec les plus grands taux de cancer semblent etre les departements avec la population agee la plus importante
#on ne remarque pas forcement de suite l effet de la pollution sur le nombre de patients atteints de cancer
#paris aurait ete plus eleve

import folium

centroid = df_carto2.geometry.centroid.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=10)

for i, row in df_carto2.iterrows():
    folium.GeoJson(
        row['geometry'],
        name=f"Dept: {row['nom']}, Pourcentage Cancer: {row['pourcentage_cancer']}",
        tooltip=f"Dept: {row['nom']}, Pourcentage Cancer: {row['pourcentage_cancer']}",
    ).add_to(m)
m


In [ ]:
base_groupe_dep = pd.read_csv('base_groupe_dep_moy_age.csv')

base_groupe_dep = base_groupe_dep.drop(['Unnamed: 0'], axis = 1)

base_groupe_dep = base_groupe_dep[~base_groupe_dep['dept'].isin(dept_manquants)]

base_groupe_dep = base_groupe_dep.merge(base_pop, how='left', left_on = 'dept', right_on = 'numero_dep').drop(['Unnamed: 0'], axis = 1)

base_groupe_dep = base_groupe_dep.set_index('dep')

num_cols_dep = base_groupe_dep.select_dtypes(include=['number']).columns
num_data_dep = base_groupe_dep[num_cols_dep]
scaler = StandardScaler()
base_standard_dep = scaler.fit_transform(num_data_dep)
base_standard_dep = pd.DataFrame(data=base_standard_dep, columns=num_cols_dep, index=base_groupe_dep.index)

In [ ]:
base_standard_dep = base_standard_dep.drop('indicatrice_Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)', axis = 1)

In [ ]:
base_standard_dep = base_standard_dep.drop(['indicatrice_Autres affections de longue durée (dont 31 et 32)',
                                            'indicatrice_Diabète',
                                            'indicatrice_Maladies inflammatoires ou rares ou VIH ou SIDA',
                                            'indicatrice_Maladies neurologiques ou dégénératives',
                                            'indicatrice_Maternité (avec ou sans pathologies)',
                                            'indicatrice_Traitements psychotropes (hors pathologies)',
                                            'indicatrice_Insuffisance rénale chronique terminale',
                                            'indicatrice_Séjours en hospitalisation complète pour prise en charge de la Covid-19',
                                           'indicatrice_Traitements du risque vasculaire (hors pathologies)',
                                           'indicatrice_Maladies psychiatriques'], axis = 1)
#on laisse tomber ces colonnes pour essayer d avoir les maladies qui en theorie devraient 
#etre causees par la pollution

In [ ]:
base_standard_dep.columns

In [ ]:
pca = PCA(n_components=2)
pca_result_dep = pca.fit_transform(base_standard_dep)
pca_df_dep = pd.DataFrame(data=pca_result_dep, columns=['PC1', 'PC2'])

# Apply Promax rotation using factor_analyzer
fa_promax = FactorAnalyzer(n_factors=2, rotation='promax')
fa_promax_result_dep = fa_promax.fit_transform(base_standard_dep)
fa_promax_df_dep = pd.DataFrame(data=fa_promax_result_dep, columns=['Factor1', 'Factor2'])

# Get loadings for Promax rotated factors
loadings_dep = fa_promax.loadings_
loading_matrix_dep = pd.DataFrame(loadings_dep, columns=['Factor1', 'Factor2'], index=base_standard_dep.columns)

# Plotting
fig, (ax_pca, ax_fa) = plt.subplots(1, 2, figsize=(16, 8))

# Plot circle for PCA
circle_pca = plt.Circle((0, 0), 1, fill=False, edgecolor='b', linestyle='dashed')
ax_pca.add_patch(circle_pca)
for i, var_pca in enumerate(pca_df_dep.columns):
    ax_pca.arrow(0, 0, pca.components_[0, i], pca.components_[1, i],
                 color='r', alpha=0.7, linewidth=0.5, head_width=0.02, head_length=0.02)
    ax_pca.text(pca.components_[0, i] * 1.15, pca.components_[1, i] * 1.15, var_pca,
                color='b', ha='center', va='center')
ax_pca.set_xlim([-1, 1])
ax_pca.set_ylim([-1, 1])
ax_pca.set_aspect('equal', 'box')
ax_pca.set_title('Circle of Correlation for PCA')

# Plot circle for Promax rotated factors
circle_fa = plt.Circle((0, 0), 1, fill=False, edgecolor='b', linestyle='dashed')
ax_fa.add_patch(circle_fa)
for i, var_fa in enumerate(loading_matrix_dep.index):
    ax_fa.arrow(0, 0, loading_matrix_dep.loc[var_fa, 'Factor1'], loading_matrix_dep.loc[var_fa, 'Factor2'],
                color='r', alpha=0.7, linewidth=0.5, head_width=0.02, head_length=0.02)
    ax_fa.text(loading_matrix_dep.loc[var_fa, 'Factor1'] * 1.15, loading_matrix_dep.loc[var_fa, 'Factor2'] * 1.15, var_fa,
               color='b', ha='center', va='center')
ax_fa.set_xlim([-1, 1])
ax_fa.set_ylim([-1, 1])
ax_fa.set_aspect('equal', 'box')
ax_fa.set_title('Circle of Correlation for Promax Rotation')

plt.show()


In [ ]:
num_clusters_range = range(1, 11)  # You can adjust the range as needed

# Fit K-Means models for different cluster numbers
inertia_values = []
for n_clusters in num_clusters_range:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(base_standard_dep)
    inertia_values.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(num_clusters_range, inertia_values, marker='o')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia (Within-cluster Sum of Squares)')
plt.show()

In [ ]:
print(fa_promax_df_dep.columns)


In [ ]:
base_standard_dep_reset = base_standard_dep.reset_index()

# Print columns of base_standard_dep_reset
print("Columns of base_standard_dep_reset:", base_standard_dep_reset.columns)

# PCA with Promax rotation
pca = PCA(n_components=2)
pca_result_bis = pca.fit_transform(base_standard_dep_reset.drop(columns=['dep']))  # Exclude 'dep' from PCA
pca_df_reduit = pd.DataFrame(data=pca_result_bis, columns=['PC1', 'PC2'])

# Promax rotation
fa = FactorAnalyzer(rotation='promax', n_factors=2)
fa.fit(base_standard_dep_reset.drop(columns=['dep']))  # Exclude 'dep' from factor analysis
rotated_loadings = fa.loadings_

# Elbow Method for determining the optimal number of clusters
wcss = []  # Within-Cluster Sum of Squares

for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(pca_df_reduit)
    wcss.append(kmeans.inertia_)

# Plot the elbow method
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# K-means clustering
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
base_standard_dep_reset['Cluster'] = kmeans.fit_predict(pca_df_reduit)

# Combine PCA results with original DataFrame, including 'dep'
df_combined = pd.concat([base_standard_dep_reset[['dep', 'Cluster']], pca_df_reduit], axis=1)

# Scatter plot with cluster labels and "dep" values using Plotly
fig = px.scatter(df_combined, x='PC1', y='PC2', color='Cluster',
                 title='PCA Clusters with Labels', labels={'Cluster': 'Cluster'},
                 color_continuous_scale='viridis', hover_name='dep')

# Show the plot
fig.show()


In [ ]:
standard_dep = base_standard_dep.merge(france, how='left', left_index=True, right_on='nom')

In [ ]:
standard_dep = standard_dep.set_index('nom')
base_standard_dep_reset = base_standard_dep_reset.set_index('dep')

In [ ]:
cluster_column = base_standard_dep_reset['Cluster']

# Add 'Cluster' column to standard_dep using .loc with values-based matching
standard_dep['Cluster'] = cluster_column.loc[standard_dep.index.values]

# Display the updated GeoDataFrame
standard_dep.head()

In [ ]:
standard_dep = gpd.GeoDataFrame(standard_dep, geometry='geometry')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
standard_dep.plot(column='Cluster', cmap='viridis', legend=True, ax=ax)
ax.set_title('Map with Clusters')
plt.show()